[SOFTEX] Residência em TIC - MCTI Futuro - IFMA 

# 3. Ciência de Dados em Python 
## 3.4. Preparação de Dados (parte II): Dados organizados (tidy data)

Este notebook é parte do material do Curso de Treinamento em IA criado pelo IFMA/DComp e financiado pelo SOFTEX, 2023.


Elaborado por Prof. Josenildo Silva (jcsilva@ifma.edu.br)



## Fonte
Este notebook é baseado no material encontrado nas referências abaixo.

Blogs
- https://medium.com/analytics-vidhya/tidying-data-in-python-78e48eb0bdfd
- https://www.jeannicholashould.com/tidy-data-in-python.html

Artigo
- Wickham, H. "Tidy Data". Journal of Statistical Software, 2014. 


# Dados Organizados

É comum que datasets sejam armazenados em vários formatos, adequados à leitura humana. 

Entretanto, estes dados apresentam dificulades para tratamento computacional. 

Uma parte importante da tarefa do analista de dados é resolver estas dificuldades produzindo um dataset **organizado** (tidy). 






As regras para um dataset ser considerado organizado são (Wickham,2014):
1. Cada variável forma uma coluna
1. Cada observação forma uma tupla, ou linha
1. Cada tipo observacional forma uma tabela (entidade)

## Exemplo de dataset desorganizado

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
pew_df = pd.read_csv('pew-untidy.csv')

In [3]:
pew_df.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


Este dataset registra a relação entre religião e salários anuais. O formato apresetando é também chamado de long form na literatura estatística. Este formato é excelente para compreensão dos relacionamentos entre variáveis, mas pode ser complicado para automatizar rotinas de análise. 

**Problema identificados**: nomes das colunas são valores de uma variável (`income`) 
    

### Exemplos de análise com dados no formato de tabela

Quantos agnosticos ganham até 50k?

In [4]:
(pew_df.query('religion == "Agnostic"').loc[:,"<$10k":"$40-50k"]).sum().sum()

278

Quantos responderam de cada religião?

In [5]:
for i in range(18):
    print(f'{pew_df.iloc[i,0]}: {pew_df.iloc[i,1:].sum()}')
    
    

Agnostic: 826
Atheist: 515
Buddhist: 411
Catholic: 8054
Don’t know/refused: 272
Evangelical Prot: 9472
Hindu: 257
Historically Black Prot: 1995
Jehovah's Witness: 215
Jewish: 682
Mainline Prot: 7470
Mormon: 581
Muslim: 116
Orthodox: 363
Other Christian: 129
Other Faiths: 449
Other World Religions: 42
Unaffiliated: 3707


## Problema 1: nome de colunas são valores de um atributo

#### Exemplo do Problema 1 com dataset PEW

In [6]:
coluna_id = pew_df.columns[0]
coluna_id

'religion'

Os valores da variável `income` estão sendo utilizados como nomes de colunas. Vamos fazer uma operação de mover estes nomes de colunas para virarem conteúdo de uma célula. O resultado vai ser uma tupla `(religiao, income, frequencia)`

In [10]:
valores_col=pew_df.columns[1:]
valores_col


Index(['<$10k', '$10-20k', '$20-30k', '$30-40k', '$40-50k', '$50-75k',
       '$75-100k', '$100-150k', '>150k', 'Don't know/refused'],
      dtype='object')

In [11]:
tidy_pew_df=pew_df.melt(id_vars=coluna_id,
                        value_vars=valores_col, 
                        var_name='income',
                        value_name='freq')
tidy_pew_df.head()

,religion,income,freq
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15


Se o parâmetro `value_vars` for omitido, todas as colunas serão utilizadas no processo. Fizemos isso de modo explícito, utilizando os nomes de todas as colunas apenas para fins didáticos. 

#### Exemplo do Problema 1 com o dataset Bilboard

In [14]:
bil_df = pd.read_csv("billboard-untidy.csv", encoding="mac_latin2")
bil_df.head(10)

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17,2000-08-26,59,52.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25,2000-03-18,83,83.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01,2000-06-24,63,45.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2000,Sisqo,Incomplete,3:52,Rock,2000-06-24,2000-08-12,77,66.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,81,54.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
id_vars = ["year",
           "artist.inverted",
           "track",
           "time",
           "genre",
           "date.entered",
           "date.peaked"]

In [16]:
bil_df_melt = df = pd.melt(frame=bil_df,
                           id_vars=id_vars, 
                           var_name="week", 
                           value_name="rank")

Remover as colunas week (melting)

In [17]:
bil_df_melt.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,x1st.week,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,x1st.week,57.0


Formatar os valores em `week`

In [18]:
# Formatting 
bil_df_melt["week"] = bil_df_melt['week'].str.extract('(\d+)', expand=False).astype(int)

In [19]:
bil_df_melt=bil_df_melt.dropna().copy()

In [20]:
bil_df_melt['rank']=bil_df_melt['rank'].astype('int')

In [22]:
bil_df_melt.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57


Criar coluna "date". A data será calculada a partir da data de entrada, somada à quantidade de semanas indicada por `week`. Isto é conseguido com a função `to_timedelta()`. No exemplo abaixo, calculamos o dia que ocorre após 5 semanas contando de 2022-01-01. Como a semana 1 não deve ser a base, subtraimos de DateOffset(weeks=1)

In [23]:
# Exemplo
pd.to_timedelta(5,unit='w')+pd.to_datetime('2022-01-01')-pd.DateOffset(weeks=1)

Timestamp('2022-01-29 00:00:00')

In [24]:
bil_df_melt['date'] = pd.to_datetime(bil_df_melt['date.entered']) + pd.to_timedelta(bil_df_melt['week'], unit='w') - pd.DateOffset(weeks=1)

In [25]:
bil_df_melt = bil_df_melt[["year", 
         "artist.inverted",
         "track",
         "time",
         "genre",
         "week",
         "rank",
         "date"]]
bil_df_melt = bil_df_melt.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])

In [26]:
bil_df_melt.head()

,year,artist.inverted,track,time,genre,week,rank,date
246,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87,2000-02-26
563,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82,2000-03-04
880,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,3,72,2000-03-11
1197,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,4,77,2000-03-18
1514,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,5,87,2000-03-25


### Exemplos de ánálise com os dados organizados

Quantos agnosticos ganham até 50k?

In [21]:
tidy_pew_df.query('religion=="Agnostic" and income in ["<$10k", "$10-20k", "$20-30k", "$30-40k","$40-50k"]')['freq'].sum()

278

Quantos em cada religião responderam?

In [29]:
a = tidy_pew_df.groupby(by=['religion']).sum(numeric_only=True)
a.sort_values(by='freq',ascending=False)[:5]
#a.reset_index().sort_values(by='freq',ascending=False)

,freq
religion,
Evangelical Prot,9472
Catholic,8054
Mainline Prot,7470
Unaffiliated,3707
Historically Black Prot,1995


Como se obeserva acima, o formato organizado permite consultas onde a condição é expressa de modo similar ao SQL. Os nomes de colunas não são mais valores. 

## Problema 2: Múltiplos tipos em uma tabela

A tabela mistrura informações de música e de rank. Como consequencia, há muita redundância na tabela. 

In [31]:
songs_cols = ["year", "artist.inverted", "track", "time", "genre"]
songs = bil_df_melt[songs_cols].drop_duplicates()
songs = songs.reset_index(drop=True)
songs["song_id"] = songs.index
songs.head()

,year,artist.inverted,track,time,genre,song_id
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,0
1,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1
2,2000,3 Doors Down,Kryptonite,3:53,Rock,2
3,2000,3 Doors Down,Loser,4:24,Rock,3
4,2000,504 Boyz,Wobble Wobble,3:35,Rap,4


In [33]:
ranks = pd.merge(bil_df_melt, songs, on=["year","artist.inverted", "track", "time", "genre"])

In [34]:
ranks.head()

,year,artist.inverted,track,time,genre,week,rank,date,song_id
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87,2000-02-26,0
1,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82,2000-03-04,0
2,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,3,72,2000-03-11,0
3,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,4,77,2000-03-18,0
4,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,5,87,2000-03-25,0


In [35]:
ranks = ranks[["song_id", "date","rank"]]
ranks.head()

,song_id,date,rank
0,0,2000-02-26,87
1,0,2000-03-04,82
2,0,2000-03-11,72
3,0,2000-03-18,77
4,0,2000-03-25,87


## Problema 3: Multiplas variáveis em uma coluna

In [37]:
tb_df = pd.read_csv('tb-untidy-sample.csv')

In [38]:
tb_df.head()

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0


In [39]:
tb_df_melt = pd.melt(tb_df, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")

In [40]:
tb_df_melt.head()

,country,year,sex_and_age,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0


In [72]:
# Extract Sex, Age lower bound and Age upper bound group
tmp_df = tb_df_melt["sex_and_age"].str.extract("(\D)(\d+)(\d{2})")    

In [73]:
# Name columns
tmp_df.columns = ["sex", "age_lower", "age_upper"]

In [74]:
# Create `age`column based on `age_lower` and `age_upper`
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

In [75]:
# Merge 
tb_df_melt = pd.concat([tb_df_melt, tmp_df], axis=1)

In [76]:
tb_df_melt

,country,year,sex_and_age,cases,sex,age_lower,age_upper,age
0,AD,2000,m014,0.0,m,0,14,0-14
1,AE,2000,m014,2.0,m,0,14,0-14
2,AF,2000,m014,52.0,m,0,14,0-14
3,AG,2000,m014,0.0,m,0,14,0-14
4,AL,2000,m014,2.0,m,0,14,0-14
...,...,...,...,...,...,...,...,...
85,AM,2000,f014,1.0,f,0,14,0-14
86,AN,2000,f014,0.0,f,0,14,0-14
87,AO,2000,f014,247.0,f,0,14,0-14
88,AR,2000,f014,121.0,f,0,14,0-14


In [77]:
# Drop unnecessary columns and rows
tb_df_melt = tb_df_melt.drop(columns=['sex_and_age',"age_lower","age_upper"], axis=1)
tb_df_melt = tb_df_melt.dropna()
tb_df_melt = tb_df_melt.sort_values(by=["country", "year", "sex", "age"],ascending=True)
tb_df_melt.head(10)

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24
20,AD,2000,1.0,m,25-34
30,AD,2000,0.0,m,35-44
40,AD,2000,0.0,m,45-54
50,AD,2000,0.0,m,55-64
81,AE,2000,3.0,f,0-14
1,AE,2000,2.0,m,0-14
11,AE,2000,4.0,m,15-24
21,AE,2000,4.0,m,25-34


## Problema 4: variável em linhas e colunas

Neste exemplos as variavéis estão armazenadas em linhas `(tmin, tmax)` e também em colunas `(days)`.

In [25]:
wea_df = pd.read_csv('weather-untidy.csv')

In [26]:
wea_df.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Extrair o dia

In [27]:
wea_df_melt = wea_df.melt(id_vars=['id','year','month','element'],var_name='day_raw')

In [28]:
wea_df_melt["day"] = wea_df_melt["day_raw"].str.extract("d(\d+)", expand=False)  


In [29]:
wea_df_melt.head()

,id,year,month,element,day_raw,value,day
0,MX17004,2010,1,tmax,d1,NaN,1
1,MX17004,2010,1,tmin,d1,NaN,1
2,MX17004,2010,2,tmax,d1,NaN,1
3,MX17004,2010,2,tmin,d1,NaN,1
4,MX17004,2010,3,tmax,d1,NaN,1


In [30]:
wea_df_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       682 non-null    object 
 1   year     682 non-null    int64  
 2   month    682 non-null    int64  
 3   element  682 non-null    object 
 4   day_raw  682 non-null    object 
 5   value    66 non-null     float64
 6   day      682 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 37.4+ KB


In [31]:
# To numeric values
wea_df_melt[["year","month","day"]] = wea_df_melt[["year","month","day"]].apply(lambda x: pd.to_numeric(x, errors='ignore'))

In [32]:
wea_df_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       682 non-null    object 
 1   year     682 non-null    int64  
 2   month    682 non-null    int64  
 3   element  682 non-null    object 
 4   day_raw  682 non-null    object 
 5   value    66 non-null     float64
 6   day      682 non-null    int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 37.4+ KB


In [33]:
def create_date_from_year_month_day(row):
    return pd.to_datetime(pd.DataFrame({'year':[row['year']],'month':[row['month']],'day':[row['day']]}),errors='ignore')
    #return datetime(year=row["year"], month=int(row["month"]), day=row["day"])

In [34]:
wea_df_melat = wea_df_melt.dropna().copy()
wea_df_melt["date"] = wea_df_melt.apply(lambda row: create_date_from_year_month_day(row), axis=1)
wea_df_melt = wea_df_melt.drop(['year',"month","day", "day_raw"], axis=1)

In [35]:

wea_df_melt

,id,element,value,date
0,MX17004,tmax,NaN,2010-01-01 00:00:00
1,MX17004,tmin,NaN,2010-01-01 00:00:00
2,MX17004,tmax,NaN,2010-02-01 00:00:00
3,MX17004,tmin,NaN,2010-02-01 00:00:00
4,MX17004,tmax,NaN,2010-03-01 00:00:00
...,...,...,...,...
677,MX17004,tmin,NaN,2010-10-31 00:00:00
678,MX17004,tmax,NaN,20101131
679,MX17004,tmin,NaN,20101131
680,MX17004,tmax,NaN,2010-12-31 00:00:00


In [36]:
# Unmelting column "element"
df = wea_df_melt.pivot(index=["id","date"], columns="element", values="value")
df.reset_index(drop=False, inplace=True)
df.dropna().sort_values(by='date')

element,id,date,tmax,tmin
319,MX17004,2010-01-30 00:00:00,27.8,14.5
12,MX17004,2010-02-02 00:00:00,27.3,14.4
23,MX17004,2010-02-03 00:00:00,24.1,14.4
111,MX17004,2010-02-11 00:00:00,29.7,13.4
243,MX17004,2010-02-23 00:00:00,29.9,10.7
46,MX17004,2010-03-05 00:00:00,32.1,14.2
101,MX17004,2010-03-10 00:00:00,34.5,16.8
167,MX17004,2010-03-16 00:00:00,31.1,17.6
289,MX17004,2010-04-27 00:00:00,36.3,16.7
290,MX17004,2010-05-27 00:00:00,33.2,18.2
